# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE7 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun
from langchain_core.tools import Tool


def helloworld(name: str) -> str:
    return f"Hello ${name}"

hello_tool = Tool(
    name = "helloworld",
    func = helloworld,
    description = "Greet the user"
)

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
    hello_tool
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [12]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [13]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?


Looking at our custom tool implementation, we have something like this;
```
hello_tool = Tool(
    name = "helloworld",
    func = helloworld,
    description = "Greet the user"
)
```
The model uses the description of the tool to determine what the tool does in an agentic workflow.



## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [14]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [15]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [16]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [17]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [18]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [19]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [20]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?
Yes, based on the langgraph docs, we do have a `graph_recursion_limit=25`. But, based on our requirements, we can either increase or deecrease this amount.

If not, how could we impose a limit to the number of cycles?
We can keep track of the aggregator value in our state, so that with each call, we can increase the value of the aggregator if it's  less than the max limit. And if it's equal or more, we'll redirect the flow to END.

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [21]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_l1PvMqPG8djHKYIbonOUiHke', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 188, 'total_tokens': 211, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtTG893iqnzAxe1gIxLvYIBd3ZLQe', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--793ec0c8-b3d6-45b1-aa2c-740430158e5c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_l1PvMqPG8djHKYIbonOUiHke', 'type': 

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [24]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7MIdHo8IFK1RlfMXkGA2jogY', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_9TgI5XlMflFrA35Q7da9aFLd', 'function': {'arguments': '{"query": "latest Tweet of the first author of QLoRA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_5iY1dSinISEsiUIzk3ZF2wps', 'function': {'arguments': '{"query": "latest Tweet of the second author of QLoRA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_0eeIMsjmWOK47kTneIiujqfs', 'function': {'arguments': '{"query": "latest Tweet of the third author of QLoRA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 113, 'prompt_tokens': 204, 'total_tokens': 317, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0,

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

---

At first, we load our task/content using the message state.
The message state is passed through the start node of our graph, that is the agent node.
The agent node sees we have tool_calls available called the 'arxiv'. It thinks that we can use this to get info regarding the QLoRA paper to get the context.

Now, we receive response from the action node. The LLM used the tool arxiv and got few context out of the QLoRA and fetched the tweets of the author.

The context is read by agent and it finds the QLoRA paper on arXiv and author tweets of the paper with the title "QLoRA: Efficient Finetuning of Quantized LLM"

# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [25]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

In [26]:
agent_chain_with_formatting.invoke({"question" : "What is RAG?"})

"RAG, or Retrieval-Augmented Generation, is an AI framework that enhances the capabilities of large language models (LLMs) by integrating an external information retrieval process. This process involves searching for and retrieving relevant information from external data sources, such as databases, documents, or web sources, before generating a response. The main goal of RAG is to produce more accurate, relevant, and up-to-date responses by grounding the model's output in external knowledge, rather than relying solely on the information contained within the model's training data. This approach helps reduce inaccuracies and ensures that the generated content is aligned with the latest available information."

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [31]:

questions = [
    "What problem does FlashAttention solve?",
    "What type of memory optimization is used in FlashAttention?",
    "Who developed the FlashAttention algorithm?",
    "How does FlashAttention differ from standard attention?",
    "Why is FlashAttention useful for large models?",
    "What are the main components of FlashAttention 2?"
]

answers = [
    {"must_mention": ["memory", "bottleneck"]},
    {"must_mention": ["tiling", "SRAM"]},
    {"must_mention": ["Tri", "Dao"]},
    {"must_mention": ["faster", "exact"]},
    {"must_mention": ["scale", "efficient"]},
    {"must_mention": ["kernel", "bandwidth"]},
]


Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [32]:
from langsmith import Client

client = Client()

dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

{'example_ids': ['89091e3f-a74d-4348-9e58-7648ddc33847',
  '30ab38a2-edbd-418d-bbc6-05d3a26a8386',
  '55411111-f9b7-4632-95e5-f6b3b663dbf2',
  'f9ad6f35-637d-43c9-ad88-6ac1bac69d51',
  'ed31fd48-7b3d-41ff-a523-941aad090408',
  '03d9da6b-0ad0-41a9-8fa7-dcd8aedfc788'],
 'count': 6}

#### ❓ Question #3:

How are the correct answers associated with the questions?

---
The correct answers are associated with the questions by their index in the list. Each question in the questions list has a corresponding dictionary in the answers list at the same position. This dictionary contains:

A "must_mention" list: words or phrases that must be present in the generated answer for it to be considered correct.

Optionally, a "sample_answer": a reference answer used for comparison or reporting.

For example:

```python
questions[2] = "Who developed the FlashAttention algorithm?"
answers[2] = {
    "must_mention": ["Tri", "Dao"],
    "sample_answer": "The FlashAttention algorithm was developed by Tri Dao and his collaborators at Stanford University."
}
```
This means that:

The correct answer to Question #3 is expected to mention both "Tri" and "Dao" to pass an automated validation check.

The association is purely positional — the i-th question maps to the i-th answer.

This setup allows easy batch evaluation by looping through zip(questions, answers) in Python.

> NOTE: Feel free to indicate if this is problematic or not

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [34]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

For our case, only one evaluation failed. Looking at the reason why it failed:

The must_mention list was:

```json
["kernel", "bandwidth"]
```
But the model's output did not include the word "kernel" — it said:

**"...based on NVIDIA's CUTLASS Library... for efficient matrix multiplication and memory operations."**

It described kernels, but never literally said the word "kernel". So:

```python
all(phrase in prediction for phrase in required)  # => False
```
Thus, it was marked incorrect.

---

So, to fix the issue, we can optimize our evaluator:

instead of `all(phrase in prediction for phrase in required)`, where we are hard checking the exact term such as ***kernel***, we need to do a semantic search with something like this: `util.cos_sim(model.encode(phrase), model.encode(text))[0][0] > 0.6`



Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [35]:
experiment_results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset_name,
    evaluators=[must_mention],
    experiment_prefix=f"Search Pipeline - Evaluation - {uuid4().hex[0:4]}",
    metadata={"version": "1.0.0"},
)

View the evaluation results for experiment: 'Search Pipeline - Evaluation - 5c74-26c28a09' at:
https://smith.langchain.com/o/d271d21b-9aba-41e9-ab3b-0c1eb6900d84/datasets/87f65c83-4760-4126-a182-7264f1d85246/compare?selectedSessions=102485dd-0bdd-4007-8bc2-45092ad40ba7




0it [00:00, ?it/s]

![image info](evaluation.png)

In [36]:
experiment_results

<ExperimentResults Search Pipeline - Evaluation - 5c74-26c28a09>

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [37]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

This cell creates a new state graph named graph_with_helpfulness_check with an agent state, then adds two nodes:

"agent" node that calls the language model (call_model)

"action" node that runs a tool (tool_node)

Essentially, it sets up a workflow where the agent can generate responses and perform tool actions.

In [ ]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

This line sets the starting point of the graph to the "agent" node — meaning the graph’s execution begins with the agent generating or processing the input first.

In [39]:
graph_with_helpfulness_check.set_entry_point("agent")

This function tool_call_or_helpful does the following:

Checks if the last message in the conversation contains any tool calls. If yes, it directs to "action" (meaning the agent should invoke a tool).

If there are more than 10 messages in the conversation, it stops by returning "END".

Otherwise, it constructs a prompt asking a GPT-4.1-mini model whether the final response is “extremely helpful” based on the initial query and final response.

The model returns "Y" for helpful or "N" for unhelpful.

If helpful, the function returns "end" to finish; if not, it returns "continue" to keep going.

In [40]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

#### 🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

The tool_call_or_helpful function guides the agent’s next action by first checking if the last message includes any tool calls; if so, it routes to execute those tools. If the conversation exceeds ten messages, it ends the interaction. Otherwise, it prompts an LLM to evaluate whether the final response is extremely helpful based on the initial query. Depending on the LLM’s answer—“Y” for helpful or “N” for not—it either ends the conversation or continues refining the response.


This code sets up the decision flow for the "agent" node in the graph_with_helpfulness_check by using the tool_call_or_helpful function to determine the next step. Depending on the function’s output, the graph either loops back to the "agent" node to continue processing, moves to the "action" node to execute a tool call, or ends the workflow if the function signals completion. This allows the graph to dynamically control the agent’s behavior based on the conversation state and helpfulness evaluation.

In [41]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

This line connects the "action" node back to the "agent" node in the graph, enabling the workflow to return to the agent after a tool action is performed. This allows the agent to process the results of the tool call and continue the conversation or take further actions.

In [42]:
graph_with_helpfulness_check.add_edge("action", "agent")

This line compiles the graph_with_helpfulness_check into a runnable agent called agent_with_helpfulness_check, preparing the graph’s nodes and edges for execution. After this, you can use the compiled agent to process inputs and follow the defined workflow with helpfulness checks and tool calls.

In [43]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

This code runs the compiled agent agent_with_helpfulness_check asynchronously on an input message asking about LoRA, Tim Dettmers, and Attention. It streams partial results (stream_mode="updates") from the agent’s nodes, printing updates as they arrive. For each chunk of output, it shows which node produced the message and the message content, allowing you to observe the agent’s step-by-step reasoning, tool calls, and final answers in real time.

In [44]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rD772m75ZviFxikjzoe21CMj', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_PHRUAJvjHyI9bPdAMeAmT4t5', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_8nllw3FyCKCRMjuDsBtwabTw', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 203, 'total_tokens': 282, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BtU

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [45]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [46]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is the process of designing and refining input prompts to effectively communicate with AI language models, such as GPT, to obtain desired responses. It involves crafting prompts that are clear, specific, and contextually appropriate to guide the model's output in a useful and accurate manner.

Prompt engineering has gained significant prominence with the rise of large language models (LLMs) like GPT-3, which were released around 2020-2021. As these models became more widely adopted, the importance of effectively interacting with them through well-designed prompts became evident, leading to the emergence of prompt engineering as a specialized skill.

Would you like me to find more detailed information or specific milestones related to the development of prompt engineering?



Retrieval-Augmented Generation (RAG) is an AI framework that enhances the capabilities of large language models (LLMs) by allowing them to retrieve and incorporate relevant external information b